## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
from intake import open_catalog


# The workers

In [2]:
nb_workers=40

In [3]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(nb_workers)
c = Client(cluster)
c

<Client: 'tls://10.36.174.49:8786' processes=0 threads=0, memory=0 B>

In [4]:
c


<Client: 'tls://10.36.174.49:8786' processes=5 threads=10, memory=42.95 GB>

In [4]:
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 40, Memory=171.80 GB


# The data

In [5]:
%%time 
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/MEOM-NEMO.yaml")
ds  = cat["eNATL60_BLBT02_SSH"].to_dask()

CPU times: user 472 ms, sys: 105 ms, total: 577 ms
Wall time: 2.08 s


In [6]:
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 11688, x: 8354, y: 4729)
Coordinates:
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 2010-01-19T00:30:00 ....
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    nav_lat               (y, x) float32 dask.array<chunksize=(240, 480), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(240, 480), meta=np.ndarray>
    sossheig              (time_counter, y, x) float32 dask.array<chunksize=(240, 240, 480), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(240, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(240, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    TimeStamp:    24/03/2019 14:43:29 +0100
    description:  ocean T grid variables
    file_name:    eNATL60-BLBT02X_1h_20100101_20100125_gridT-2D_20100119-2010...
    ibegin:       0
    jbegin:       0
    name:         /scratch/tmp/5251284/eNATL60-BLBT02X_1h_20100101_20100125_g...
    ni:           8354
    nj:           10
    timeStamp:    2019-Mar-24 06:31:35 GMT
    title:        ocean T grid variables
    uuid:         80877374-74d2-468e-9fc6-d8cae8009094

In [7]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 116 ms, sys: 7.68 ms, total: 123 ms
Wall time: 120 ms


In [8]:
mean

<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
dask.array<mean_agg-aggregate, shape=(4729, 8354), dtype=float32, chunksize=(240, 480), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x

In [9]:
%time mean.load()

CPU times: user 5.38 s, sys: 561 ms, total: 5.95 s
Wall time: 4min 17s


<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Dimensions without coordinates: y, x

In [10]:
%time mean_space=ds.sossheig.mean(dim='x').mean(dim='y')

CPU times: user 131 ms, sys: 993 µs, total: 132 ms
Wall time: 129 ms


In [11]:
mean_space

<xarray.DataArray 'sossheig' (time_counter: 11688)>
dask.array<mean_agg-aggregate, shape=(11688,), dtype=float32, chunksize=(240,), chunktype=numpy.ndarray>
Coordinates:
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-01-19T00:30:00 ... 2009-09-17T23:30:00

In [12]:
%time mean_space.load()

CPU times: user 5.84 s, sys: 308 ms, total: 6.14 s
Wall time: 8min 50s


<xarray.DataArray 'sossheig' (time_counter: 11688)>
array([-0.24003598, -0.23536953, -0.23589204, ..., -0.20522492,
       -0.21433835, -0.23148733], dtype=float32)
Coordinates:
    time_centered  (time_counter) datetime64[ns] 2010-01-19T00:30:00 ... 2009-09-17T23:30:00
  * time_counter   (time_counter) datetime64[ns] 2010-01-19T00:30:00 ... 2009-09-17T23:30:00

In [19]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
